In [ ]:
!pip install --upgrade pip
!pip install -qU pypdf
!pip install -qU langchain_community
!pip install -qU tiktoken
!pip install langgraph
!pip install -qU langchain-core

  Using cached pip-25.0.1-py3-none-any.whl.metadata (3.7 kB)
Using cached pip-25.0.1-py3-none-any.whl (1.8 MB)



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: To modify pip, please run the following command:
C:\Users\isharah\AppData\Local\Programs\Python\Python313\python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [18]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Load the PDF
loader = PyPDFLoader('./sample.pdf')
pages = []

# Each page is a Document object
async for page in loader.alazy_load():
   pages.append(page)

# Split the text into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
# Pass the list of Document objects
all_splits = text_splitter.split_documents(pages)

# Print first 5 chunks
print(all_splits[:5])

[Document(metadata={'producer': 'Microsoft® Word 2016', 'creator': 'Microsoft® Word 2016', 'creationdate': '2025-03-24T09:59:08+05:30', 'author': 'isu1', 'moddate': '2025-03-24T09:59:08+05:30', 'source': './firewall.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1'}, page_content="SEYLAN BANK PLC – FIRWALL CHANGE REQUEST \nClassification | Restricted \n   \n \n \n \n \n \n \n \n \n \nPlease Specify full detail of the change with business requirement  \n \n \n \n \n \n \n Source Address / \nSubnet Mask \nDestination Address / \nSubnet Mask  \nDestination  \nPort \nAction: Deny / \nPermit \nRule: Add / \nRemove / Modify Description \n1 139.59.118.181 203.115.28.249 2250 Permit Add LIVE \n \n \n \n \n \nRequester's Name and Staff No Requester's Email Request Type: \nNew / Additional / Delete Date of Request \nRajitha Jayanetti rajithaj@seylan.lk NEW 28/05/2024 \nPermanent/Temporary \n(Period) \nNDA with third \nparty (Yes / No) \nRequester's Mobile Phone& \nExtension Requester's Signat

In [ ]:
!pip install -qU langchain-huggingface

Create Embeddings

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")


Create an In-Memory Vector Store

In [ ]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)

In [ ]:
document_ids = vector_store.add_documents(documents=all_splits)

In [ ]:
!pip install -qU "langchain[groq]"

In [ ]:
import getpass
import os

os.environ.get("GROQ_API_KEY")
from langchain.chat_models import init_chat_model

llm = init_chat_model("llama3-8b-8192", model_provider="groq")

Then we can invoke the Chat to ask the questions from our PDF

In [ ]:
from langchain import hub
from langchain_core.documents import Document
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict

prompt = hub.pull("rlm/rag-prompt")

# Define state for application
class State(TypedDict):
   question: str
   context: List[Document]
   answer: str

# Define application steps
def retrieve(state: State):
   retrieved_docs = vector_store.similarity_search(state["question"])
   return {"context": retrieved_docs}

def generate(state: State):
   docs_content = "\n\n".join(doc.page_content for doc in state["context"])
   messages = prompt.invoke({"question": state["question"], "context": docs_content})
   response = llm.invoke(messages)
   return {"answer": response.content}

# Compile application and test
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()


In [ ]:
response = graph.invoke({"question": "What's your name ?"})
print(response["answer"])